In [1]:
import mhcflurry, seaborn, numpy, pandas, pickle, sklearn, collections, scipy, time
import mhcflurry.data
import mhcflurry.imputation
import fancyimpute, locale

import sklearn.metrics
import sklearn.cross_validation

Using Theano backend.
/Users/tim/venvs/analysis-venv-2.7/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


Couldn't import dot_parser, loading of dot files will not be possible.


/Users/tim/venvs/analysis-venv-2.7/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [7]:
max_ic50 = 50000
data_dir = "/Users/iskander/code/mhcflurry/"
data_dir = "/Users/tim/sinai/git/mhcflurry/"

In [2]:
all_train_data = mhcflurry.data.load_allele_datasets(data_dir + "bdata.2009.mhci.public.1.txt")
all_validation_data = mhcflurry.data.load_allele_datasets(data_dir + "bdata.2013.mhci.public.blind.1.txt")


In [17]:
validation_df = pandas.read_csv("../data/combined_test_BLIND_dataset_from_kim2013.csv")
validation_df

,allele,peptide,length,meas,netmhc,netmhcpan,smmpmbec_cpp
0,H-2-DB,AAACNVATA,9,657.657837,154.881662,711.213514,438.530698
1,H-2-DB,AAFEFVYV,8,30831.879502,6456.542290,785.235635,10351.421667
2,H-2-DB,AAFVNDYSL,9,77.446180,17.458222,7.516229,28.054336
3,H-2-DB,AAIANQAAV,9,1.999862,9.638290,9.749896,25.703958
4,H-2-DB,AAIANQAVV,9,1.517050,8.550667,8.336812,28.773984
5,H-2-DB,AAIENYVRF,9,37.844258,252.348077,114.815362,187.068214
6,H-2-DB,AAINFITTM,9,3.155005,199.986187,389.045145,200.909281
7,H-2-DB,AAIPAPPPI,9,3243.396173,1059.253725,493.173804,295.120923
8,H-2-DB,AAKLNRPPL,9,654.636174,66.374307,77.268059,38.459178
9,H-2-DB,AALDMVDAL,9,229.614865,547.015963,597.035287,225.423921


In [18]:
'''
(validation_df, peptide_col_name) = mhcflurry.data.load_dataframe(
        filename="../data/combined_test_BLIND_dataset_from_kim2013.csv",
        max_ic50=max_ic50,
        sep=None,
        species_column_name="species",
        allele_column_name="mhc",
        peptide_column_name=None,
        filter_peptide_length=None,
        ic50_column_name="meas",
        only_human=False)
validation_df
'''

'\n(validation_df, peptide_col_name) = mhcflurry.data.load_dataframe(\n        filename="../data/combined_test_BLIND_dataset_from_kim2013.csv",\n        max_ic50=max_ic50,\n        sep=None,\n        species_column_name="species",\n        allele_column_name="mhc",\n        peptide_column_name=None,\n        filter_peptide_length=None,\n        ic50_column_name="meas",\n        only_human=False)\nvalidation_df\n'

In [5]:
alleles = [
    "HLA-A0201",
    "HLA-B2705",
    "HLA-A0250",
    'HLA-B7301',
    # "HLA-A0301",
    # "HLA-A0203",
    # "HLA-A2602",
    # "HLA-A2603",
    # 'HLA-B7301',
]

In [6]:
#sorted(dict((allele, len(all_train_data[allele].Y)) for allele in all_train_data).items(), key=lambda pair: -1*pair[1])

In [3]:
dropout_probabilities = [0.0, 0.1, 0.5]

embedding_output_dims = [16, 32, 64, 128]
#embedding_output_dims = [4, 32]

#layer_sizes = [[4], [8], [16], [64], [128]]
layer_sizes_list = [[16], [64], [100], [128]]

activations = ["tanh"]

models_params_list = []
for dropout_probability in dropout_probabilities:
    for embedding_output_dim in embedding_output_dims:
        for layer_sizes in layer_sizes_list:
            for activation in activations:
                models_params_list.append(dict(
                    dropout_probability=dropout_probability,  
                    embedding_output_dim=embedding_output_dim,
                    layer_sizes=layer_sizes,
                    activation=activation))

print("%d models" % len(models_params_list))
models_params_explored = set.union(*[set(x) for x in models_params_list])
models_params_explored


48 models


{'activation', 'dropout_probability', 'embedding_output_dim', 'layer_sizes'}

In [40]:
def log_to_ic50(log_value):
        """
        Convert neural network output to IC50 values between 0.0 and
        self.max_ic50 (typically 5000, 20000 or 50000)
        """
        return max_ic50 ** (1.0 - log_value)

def make_scores(ic50_y, ic50_y_pred, sample_weight=None, threshold_nm=500):     
    y_pred = mhcflurry.common.ic50_to_regression_target(ic50_y_pred, max_ic50)
    try:
        auc = sklearn.metrics.roc_auc_score(ic50_y <= threshold_nm, y_pred, sample_weight=sample_weight)
    except ValueError:
        auc = numpy.nan
    try:
        f1 = sklearn.metrics.f1_score(ic50_y <= threshold_nm, ic50_y_pred <= threshold_nm, sample_weight=sample_weight)
    except ValueError:
        f1 = numpy.nan
    try:
        tau = scipy.stats.kendalltau(ic50_y_pred, ic50_y)[0]
    except ValueError:
        tau = numpy.nan
    
    return dict(
        auc=auc,
        f1=f1,
        tau=tau,
    )    

def mean_with_std(grouped_column, decimals=3):
    pattern = "%%0.%df" % decimals
    return pandas.Series([
        (pattern + " +/ " + pattern) % (m, s) if not pandas.isnull(s) else pattern % m
        for (m, s) in zip(grouped_column.mean(), grouped_column.std())
    ], index = grouped_column.mean().index)

def allele_data_to_df(data):
    d = data._asdict()
    d["X_index"] = [x for x in d["X_index"]]
    d["X_binary"] = [x for x in d["X_binary"]]
    df = pandas.DataFrame(d).set_index('peptides')
    return df

def make_2d_array(thing):
    return numpy.array([list(x) for x in thing])

def df_to_allele_data(df):
    d = dict((col, df[col].values) for col in df)
    d["X_index"] = make_2d_array(d["X_index"])
    (d["max_ic50"],) = list(df.max_ic50.unique())
    return mhcflurry.data.AlleleData(peptides = df.index.values, **d)


In [43]:
scores_df = collections.defaultdict(list)
predictors = validation_df.columns[4:]

for (allele, grouped) in validation_df.groupby("allele"):
    scores_df["allele"].append(allele)
    scores_df["test_size"].append(len(grouped.meas))
    for predictor in predictors:
        scores = make_scores(grouped.meas, grouped[predictor])
        for (key, value) in scores.items():
            scores_df["%s_%s" % (predictor, key)].append(value)
    
    

scores_df = pandas.DataFrame(scores_df)
scores_df.sort("test_size", ascending=False, inplace=True)
scores_df

,allele,netmhc_auc,netmhc_f1,netmhc_tau,netmhcpan_auc,netmhcpan_f1,netmhcpan_tau,smmpmbec_cpp_auc,smmpmbec_cpp_f1,smmpmbec_cpp_tau,test_size
4,HLA-A0201,0.932234,0.884336,0.635498,0.930479,0.880963,0.637338,0.927358,0.885121,0.626224,2126
13,HLA-A2601,0.930840,0.541176,0.436646,0.942279,0.640000,0.451666,0.927339,0.603053,0.422900,1333
27,HLA-B0801,0.946187,0.776053,0.629519,0.942822,0.701671,0.606762,0.940555,0.760000,0.618953,940
46,HLA-B5101,0.919175,0.428571,0.365307,0.948695,0.610169,0.406973,0.927621,0.395604,0.344534,854
49,HLA-B5701,0.899092,0.741840,0.484975,0.883077,0.642623,0.460133,0.929923,0.709091,0.569615,815
26,HLA-B0702,0.916126,0.869383,0.606014,0.913527,0.855011,0.604720,0.897928,0.857719,0.563960,813
8,HLA-A0301,0.935154,0.900621,0.629236,0.927287,0.885106,0.611240,0.933966,0.897275,0.610891,811
19,HLA-A3101,0.862606,0.833958,0.525026,0.859261,0.822335,0.528941,0.851270,0.837037,0.502586,724
9,HLA-A1101,0.949252,0.883402,0.632199,0.945065,0.887671,0.625794,0.944205,0.882514,0.618388,723
3,HLA-A0101,0.915334,0.619565,0.524866,0.894895,0.594286,0.498767,0.832665,0.437811,0.428064,696
